# Format

In [1]:
import os
import re
import json
from pprint import pprint
from bs4 import BeautifulSoup
from bs4.element import Comment
import sys

import nltk
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.chunk import conlltags2tree, tree2conlltags
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import RegexpTokenizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

import gensim
from pprint import pprint

from stop_words import get_stop_words

import codecs
import os
import time

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('stopwords')

import spacy
import pandas as pd
import itertools as it
import en_core_web_sm

import spacy
nlp = spacy.load('en')

[nltk_data] Downloading package punkt to /home/goodgame/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/goodgame/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /home/goodgame/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/goodgame/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/goodgame/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/goodgame/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import codecs
import boto3
# from collections import defaultdict
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import LsiModel

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle

In [3]:
s3 = boto3.resource('s3')

In [4]:
# Raghu's code to pull down jobs
BUCKET_NAME = 'tech-salary-project'

In [8]:
# %%time

# bucket = s3.Bucket(BUCKET_NAME)
# all_job_titles = []
# all_job_ids = []
# all_summaries = []
# for o in bucket.objects.all():
#     if o.key.startswith('salaries'):
#         continue
    
#     object = bucket.Object(o.key)
#     try:
#         lines = object.get()['Body'].read().decode('utf-8').splitlines()
#         for line in lines:
#             d = json.loads(line)
            
#             title = d['title']
#             jid = d['id']
#             summary = d['summary']
            
#             all_job_ids.append(jid)
#             all_job_titles.append(title)
#             all_summaries.append(summary)
#     except Exception as e:
#         continue

In [9]:
print(len(set(all_summaries)))

119326


In [3]:
%%time

# Put dataset in memory
directory = os.fsencode('../local_data/')

original_texts = []

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    full_file = '../local_data/' + str(filename)
    with open(full_file, 'r') as infile:
        text = infile.read()
        original_texts.append(text)

CPU times: user 17.4 s, sys: 13.2 s, total: 30.6 s
Wall time: 26min 57s


In [22]:
%%time
# Load in Stackoverflow data

stackoverflow_raw = []

def clean_text(text):
    cleanr = re.compile(r'\bamp\b')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

with open('../stackoverflow_data/stack_parsed.txt', 'r') as infile:
    for line in infile:
        line = line.strip()
        line = clean_text(line)
        if line == '':
            continue
        stackoverflow_raw.append(line)

# Size of list in bytes
print(sys.getsizeof(stackoverflow_raw))
print(len(stackoverflow_raw))

4290016
519155
CPU times: user 11.7 s, sys: 325 ms, total: 12 s
Wall time: 14.9 s


In [8]:
print("Length of job data list:", len(original_texts))
print("Length of Stackoverflow data list:", len(stackoverflow_raw))
all_data = original_texts + stackoverflow_raw
print("Combined dataset length:", len(all_data))

print("Size of total dataset in MB:", sys.getsizeof(all_data)/1000000)

Length of job data list: 106373
Length of Stackoverflow data list: 519155
Combined dataset length: 625528
Size of total dataset in MB: 5.004288


In [23]:
%%time
lower_stackoverflow = [item.lower().strip() for item in stackoverflow_raw]

CPU times: user 404 ms, sys: 236 ms, total: 640 ms
Wall time: 638 ms


# Alternate approach, 16APR18: Manually replace bigrams.

In [26]:
# subset = lower_data[::10000]
# print("Items in subset:",len(subset))

Items in subset: 63


In [26]:
# Load pickled skill dictionary from disk. I generated it from prior work.
with open('skill_bigram_dict.pkl', 'rb') as handle:
    skill_dict = pickle.load(handle)
print("machine learning becomes", skill_dict['machine learning'])

machine learning becomes machine_learning


In [24]:
def multiple_replace(dict, text):
    '''
    Function that uses regex to replace terms.
    Input is a dictionary of terms to switch, plus
    a string.
    '''
    # Create a regular expression  from the dictionary keys
    regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))

    # For each match, look-up corresponding value in dictionary
    return regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text) 

In [27]:
%%time
# new_data = []
# for text in lower_data:
#     new_data.append(multiple_replace(skill_dict, text))

new_stackoverflow = []
for text in lower_stackoverflow:
    new_stackoverflow.append(multiple_replace(skill_dict, text))

CPU times: user 4min 56s, sys: 159 ms, total: 4min 56s
Wall time: 4min 56s


In [28]:
# Write list to file as a pickled object, to save time later
with open('../models_12apr/stackoverflow_data_skill_bigrams_list.pkl', 'wb') as f:
    pickle.dump(new_stackoverflow, f)

In [3]:
# Load back into memory
with open('../models_12apr/all_lowercase_data_skill_bigrams_list.pkl', 'rb') as handle:
    new_data = pickle.load(handle)

In [21]:
%%time
# for index, item in enumerate(new_data):
#     if 'getelementbyid' in item:
#         print("Found!", index, item)

# print(lemmatized_sentence_corpus(new_data[106374].ipynb_checkpoints/ipynb_checkpoints/
                                 
# for sentence in lemmatized_sentence_corpus(new_data[106374:106375]):
#     print(sentence + '\n')

with open('../models_12apr/jds_skill_bigrams_concat_parsed.txt', 'r') as infile:
    for index,line in enumerate(infile[4000000:]):
        if 'while apply opacity to a form should -PRON- use a decimal or a double value' in line:
            print('Found!')
            print(index, line)

TypeError: '_io.TextIOWrapper' object is not subscriptable

#### Note: All texts are lowercase at this point.

In [29]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    12APR: adding numbers to this
    """
    
    return token.is_punct or token.is_space or token.is_digit

def line_review(filename):
    """
    SRG: modified for a list
    generator function to read in text from the file
    and un-escape the original line breaks in the text
    """
    
    for text in filename:
        yield text.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_text in nlp.pipe(line_review(filename),
                                  batch_size=5000, n_threads=24):
        
        for sent in parsed_text.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [30]:
%%time

with codecs.open('../models_12apr/jds_skill_bigrams_concat_parsed_stackoverflow.txt', 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(new_stackoverflow):
        f.write(sentence + '\n')

CPU times: user 1d 1h 16min 47s, sys: 1d 13h 27min 12s, total: 2d 14h 44min
Wall time: 3h 40min 26s


In [ ]:
# Took 5 hrs 37 minutes last time.
# Started: 3:30 PM; ETA is 9 PM.

In [15]:
unigram_sentences = LineSentence('../models_05apr/jds_concat_parsed.txt')

In [16]:
# print some examples
for unigram_sentence in it.islice(unigram_sentences, 230, 240):
    print(u' '.join(unigram_sentence))
    print(u'')

522690 disney

30 + day ago

the project manager be experience in the use of project management tool and principle who can manage activity for cross functional team within technical operations

the project manager will ensure the project be state of the art plan and document stay on track and be within the expected budget and timeline

the project manager work closely with the strategic program lead to guide the team in develop project and program plan which bring the overall program successfully through the lifecycle process and be in alignment with product strategy management and health authorities’ requirement

this role will include 50 team management and 50 project management include oversee project activity and ensure alignment with global strategy identify risk and develop mitigation plan establish team goal milestone update progress against milestone and communicate with stakeholder and senior management regard progress of project

responsibilities

support

right

first time l

In [17]:
%%time

bigram_model = Phrases(unigram_sentences)
bigram_model.save('../models_05apr/bigram_model')

CPU times: user 4min 17s, sys: 4.29 s, total: 4min 22s
Wall time: 4min 23s


In [4]:
# load the finished model from disk
bigram_model = Phrases.load('../models_05apr/bigram_model')

In [19]:
%%time

with codecs.open('../models_05apr/bigram_sentences.txt', 'w', encoding='utf_8') as f:
    for unigram_sentence in unigram_sentences:
        bigram_sentence = u' '.join(bigram_model[unigram_sentence])
        f.write(bigram_sentence + '\n')

/usr/local/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 9min 9s, sys: 1.12 s, total: 9min 10s
Wall time: 9min 11s


In [9]:
bigram_sentences = LineSentence('../models_05apr/bigram_sentences.txt')

In [11]:
# # print examples; certain bigrams are underlined
# for bigram_sentence in it.islice(bigram_sentences, 240, 250):
#     print(u' '.join(bigram_sentence))
#     print(u'')

In [12]:
%%time
trigram_model = Phrases(bigram_sentences)
trigram_model.save('../models_05apr/trigram_model')

CPU times: user 4min 45s, sys: 4.68 s, total: 4min 50s
Wall time: 4min 48s


In [13]:
# load the finished model from disk
trigram_model = Phrases.load('../models_05apr/trigram_model')

In [14]:
%%time

with codecs.open('../models_05apr/trigram_sentences.txt', 'w', encoding='utf_8') as f:
    for bigram_sentence in bigram_sentences:
        trigram_sentence = u' '.join(trigram_model[bigram_sentence])
        f.write(trigram_sentence + '\n')

/home/goodgame/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 9min 55s, sys: 3.09 s, total: 9min 59s
Wall time: 9min 59s


In [15]:
trigram_sentences = LineSentence('../models_05apr/trigram_sentences.txt')

In [16]:
for trigram_sentence in it.islice(trigram_sentences, 240, 250):
    print(u' '.join(trigram_sentence))
    print(u'')

establishes and maintain information sharing and communication structure e.g. sharepoint with current version of meeting_agenda_minute and presentation_material and deliverable draft and final

support

the right first time lead in the execution of program by translation of define strategy into team goal and milestone in alignment with regulation

owns program documentation include program plan and schedule meet agenda and minute action item dashboard goal and milestone risk register mitigation_plan decision log etc

resolving project constraint and prioritization of task in alignment with the program

facilitate the use of project management tool to achieve a balance datum drive and risk base decision_making process definition of mitigation

and/or contingency_plan executes

scenario planning and risk_assessment develop option for the team ensures project progress and upcoming deliverable be understand by the team by use pm tool

holds team_member accountable for on time completion of

In [17]:
# NLTK stopwords format is not a list; use this version
stopwords = get_stop_words('english')
additional_stopwords = ["\'s", 'or']
stopwords = stopwords + additional_stopwords
print(stopwords)

['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's", 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself', "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs', 'them', 'themselves', 't

#### Final preprocessing

In [23]:
%%time

with codecs.open('../models_05apr/final_sentences.txt', 'w', encoding='utf_8') as f:
    for parsed_review in nlp.pipe(line_review(all_data),
                                  batch_size=5000, n_threads=24):
        # lemmatize the text, removing punctuation and whitespace
        unigram_review = [token.lemma_ for token in parsed_review
                          if not punct_space(token)]

        # apply the first-order and second-order phrase models
        bigram_review = bigram_model[unigram_review]
        trigram_review = trigram_model[bigram_review]

        # remove any remaining stopwords
        trigram_review = [term for term in trigram_review
                          if term not in stopwords]

        # write the transformed review as a line in the new file
        trigram_review = u' '.join(trigram_review)
        f.write(trigram_review + '\n')

/home/goodgame/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 1d 22h 31min 16s, sys: 2d 18h 36min 50s, total: 4d 17h 8min 6s
Wall time: 7h 53min 4s


In [30]:
!wc -l ../models_05apr/final_sentences.txt

625528 ../models_05apr/final_sentences.txt


In [24]:
# probably another memory issue.

In [28]:
%%time

trigram_reviews = LineSentence('../models_05apr/final_sentences.txt')

# learn the dictionary by iterating over all of the reviews
trigram_dictionary = Dictionary(trigram_reviews)

# filter tokens that are very rare or too common from
# the dictionary (filter_extremes) and reassign integer ids (compactify)
trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
trigram_dictionary.compactify()

trigram_dictionary.save('../models_05apr/trigram_dictionary.dict')
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load('../models_05apr/trigram_dictionary.dict')

CPU times: user 1min 27s, sys: 3.94 s, total: 1min 31s
Wall time: 1min 31s


In [26]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [31]:
%%time

# generate bag-of-words representations for all JDs and save them as a matrix
MmCorpus.serialize('../models_05apr/trigram_corpus.mm',
                   trigram_bow_generator('../models_05apr/final_sentences.txt'))

CPU times: user 1min 54s, sys: 0 ns, total: 1min 54s
Wall time: 1min 54s
